<a href="https://colab.research.google.com/github/qrak/3commas-TakeProfit-Update/blob/master/lightningmodule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytorch-lightning

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


class LSTMNet(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, output_size)
        self.loss = nn.MSELoss()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout1(out)
        out = self.fc1(out[:, -1, :])
        out = self.dropout2(out)
        out = self.fc2(out)
        return out

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs.unsqueeze(1))
        loss = self.loss(outputs, labels.view(-1, 1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs.unsqueeze(1))
        loss = self.loss(outputs, labels.view(-1, 1))
        self.log('val_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs.unsqueeze(1))
        loss = self.loss(outputs, labels.view(-1, 1))
        self.log('test_loss', loss)

if __name__ == '__main__':

    # Load data into dataframe
    df = pd.read_csv('BTC_USDT_5m_2015-02-01_now_binance.csv')
    df = df.dropna()
    scaler = StandardScaler()
    # Extract the target variable (close price)
    y = df.iloc[:, 4].values

    # Extract the input features
    X = df.iloc[:, 1:4].values
    X = scaler.fit_transform(X)
    # Perform feature selection
    k = 3  # Number of features to select
    selector = SelectKBest(f_regression, k=k)
    X = selector.fit_transform(X, y)
    # Split the data into training, validation, and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, shuffle=False)
    # Move the data to the specified device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
    y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
    # Define the number of folds for cross-validation
    num_folds = 5

    # Define the hyperparameters to search over
    input_sizes = [X.shape[1]]
    hidden_sizes = [16, 32, 64]
    num_layers_list = [2, 4, 6]
    dropout_sizes = [0.1]

    num_epochs = 100
    best_val_loss = float('inf')
    best_model = None
    train_loader = None
    optimizer = None
    criterion = None
    val_loader = None
    best_hyperparams = None
    for input_size in input_sizes:
        for hidden_size in hidden_sizes:
            for num_layers in num_layers_list:
                for dropout_size in dropout_sizes:
                    print(f"Training model with input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, dropout={dropout_size}")
                    val_losses = []
                    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
                    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_val)):
                        print(f"Fold {fold + 1}/{num_folds}")
                        # Split the data into training and validation sets for this fold
                        X_fold_train, y_fold_train = X_train_val[train_idx], y_train_val[train_idx]
                        X_fold_val, y_fold_val = X_train_val[val_idx], y_train_val[val_idx]

                        # Create the PyTorch datasets and data loaders
                        train_dataset = TensorDataset(torch.tensor(X_fold_train, dtype=torch.float32).to(device),
                                                      torch.tensor(y_fold_train, dtype=torch.float32).to(device))
                        val_dataset = TensorDataset(torch.tensor(X_fold_val, dtype=torch.float32).to(device),
                                                    torch.tensor(y_fold_val, dtype=torch.float32).to(device))
                        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=False)
                        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=False)
                        # Create the model, loss function, and optimizer
                        model = LSTMNet(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                                        output_size=1, dropout=dropout_size).to(device)
                        optimizer = optim.Adam(model.parameters(), lr=1e-3)
                        criterion = nn.MSELoss()
                        # Initialize the EarlyStopping callback
                        early_stopping = pl.callbacks.EarlyStopping(patience=10, monitor='val_loss')

                        # Train the model
                        trainer = pl.Trainer(max_epochs=num_epochs, gpus=1, callbacks=[early_stopping])
                        trainer.fit(model, train_loader, val_loader)

                        # Evaluate the model on the validation set for this fold
                        val_loss = trainer.validate(val_loader)[0]['val_loss']
                        val_losses.append(val_loss)

                        mean_val_loss = np.mean(val_losses[:fold + 1])
                        print(f"Fold {fold + 1}/{num_folds}, Mean Validation Loss: {mean_val_loss:.6f}")

                        # Check if this model has a lower validation loss than the previous best model
                        if mean_val_loss < best_val_loss:
                            best_val_loss = mean_val_loss
                            best_model = model
                            best_hyperparams = (input_size, hidden_size, num_layers, dropout_size)

    # Train the best model on the full training set and evaluate it on the test set
    early_stopping = pl.callbacks.EarlyStopping(patience=10, monitor='val_loss')
    trainer = pl.Trainer(max_epochs=num_epochs, gpus=1, callbacks=[early_stopping])
    trainer.fit(best_model, train_loader, val_loader)

    # Evaluate the best model on the test set
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=8, shuffle=False)
    test_loss = trainer.test(best_model, test_loader)[0]['test_loss']

    print(f"Best hyperparameters: input_size={best_hyperparams[0]}, hidden_size={best_hyperparams[1]}, num_layers={best_hyperparams[2]}, dropout={best_hyperparams[3]}")
    print(f"Validation loss: {best_val_loss:.6f}")
    print(f"Test loss: {test_loss:.6f}")

    # Save the best model with the best hyperparameters
    torch.save(best_model.state_dict(), "best_model.pt")



Training model with input_size=3, hidden_size=16, num_layers=2, dropout=0.1
Fold 1/5


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


MisconfigurationException: ignored